In [1]:
#XGBoost를 이용한 유방암 데이터 분류

import xgboost as xgb
from xgboost import plot_importance
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [3]:
import warnings

#경고 메시지가 출력되지 않도록 설정
warnings.filterwarnings('ignore')

In [4]:
#위스콘신 유방암 데이터를 읽어서 dataset에 저장
dataset = load_breast_cancer()

In [5]:
#위스콘신 유방암 데이터의 독립변수를 X_features에 대입
X_features = dataset.data

#위스콘신 유방암 데이터의 종속변수를 y_label에 대입
y_label = dataset.target

#위스콘신 유방암 데이터의 컬럼명을 feature_names에 저장
feature_names = dataset.feature_names

#X_features를 데이터프레임으로 생성
cancer_df = pd.DataFrame(data = X_features)

#cancer_df의 컬럼명에 feature_names 대입
cancer_df.columns = feature_names

#cancer_df 데이터프레임의 target 컬럼에 y_label 컬럼의 값 대입
cancer_df['target'] = y_label
cancer_df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [6]:
#cancer_df에서 null값이 저장되어 있는지 여부를 True/False로 출력
cancer_df.isnull()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
565,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
566,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
567,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
#각 컬럼별로 합을 계산
cancer_df.isnull().sum()

mean radius                0
mean texture               0
mean perimeter             0
mean area                  0
mean smoothness            0
mean compactness           0
mean concavity             0
mean concave points        0
mean symmetry              0
mean fractal dimension     0
radius error               0
texture error              0
perimeter error            0
area error                 0
smoothness error           0
compactness error          0
concavity error            0
concave points error       0
symmetry error             0
fractal dimension error    0
worst radius               0
worst texture              0
worst perimeter            0
worst area                 0
worst smoothness           0
worst compactness          0
worst concavity            0
worst concave points       0
worst symmetry             0
worst fractal dimension    0
target                     0
dtype: int64

In [8]:
#target 컬럼의 빈도수 출력
cancer_df['target'].value_counts()

target
1    357
0    212
Name: count, dtype: int64

In [9]:
#전체 데이터를 8:2 비율로 학습:테스트용 데이터로 추출
#같은 데이터를 학습데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X_features, y_label,
                                                    test_size = 0.2, random_state = 156)

In [10]:
from xgboost import XGBClassifier

#XGBoost를 구성하는 트리의 개수를 10개로 객체를 생성하여 리턴
xgboost = XGBClassifier(n_estimators = 10)

In [12]:
#XGBoost의 알고리즘을 이용하여 트리 생성
xgboost.fit(X_train, y_train,
            eval_set = [(X_test, y_test)])

[0]	validation_0-logloss:0.44605
[1]	validation_0-logloss:0.33984
[2]	validation_0-logloss:0.27318
[3]	validation_0-logloss:0.23087
[4]	validation_0-logloss:0.19677
[5]	validation_0-logloss:0.17906
[6]	validation_0-logloss:0.15970
[7]	validation_0-logloss:0.14626
[8]	validation_0-logloss:0.13532
[9]	validation_0-logloss:0.12763


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=10, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [13]:
#X_test 데이터를 이용하여 예측 결과 리턴
pred = xgboost.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score

In [15]:
confusion_matrix(y_test, pred)

array([[33,  4],
       [ 3, 74]], dtype=int64)

In [16]:
#실제 데이터 y_test와 예측데이터 pred의 정확도 출력
accuracy_score(y_test, pred)

0.9385964912280702

In [17]:
#실제 데이터 y_test와 예측데이터 pred의 precision 출력
precision_score(y_test, pred)

0.9487179487179487

In [18]:
#실제 데이터 y_test와 예측데이터 pred의 recall 출력
recall_score(y_test, pred)

0.961038961038961

In [19]:
#실제 데이터 y_test와 예측데이터 pred의 f1 score 출력
f1_score(y_test, pred)

0.9548387096774194